# SGR Flow Curve: Viscosity and the Glass Transition

## Learning Objectives

1. Fit SGR (Soft Glassy Rheology) models to real emulsion flow curves
2. Understand the noise temperature parameter `x` and its physical meaning
3. Classify phase regimes: glass (x < 1), power-law fluid (1 ≤ x < 2), Newtonian (x ≥ 2)
4. Detect shear banding instabilities in concentrated emulsions
5. Compare SGRConventional vs SGRGeneric (thermodynamically consistent)
6. Use NLSQ warm-start for efficient Bayesian inference with NUTS

## Prerequisites

- Basic RheoJAX usage (basic/01_quick_start.ipynb)
- Bayesian inference fundamentals (bayesian/01_bayesian_basics.ipynb)

## Runtime

- Fast demo (NUM_CHAINS=1, NUM_SAMPLES=500): ~2 minutes
- Full run (NUM_CHAINS=4, NUM_SAMPLES=2000): ~5 minutes

## 1. Setup

In [ ]:
# Google Colab setup
import sys

IN_COLAB = "google.colab" in sys.modules
if IN_COLAB:
    %pip install -q rheojax
    import os
    os.environ["JAX_ENABLE_X64"] = "true"
    print("RheoJAX installed successfully.")

In [ ]:
# Imports
%matplotlib inline
import json
import os
import time
import warnings

import arviz as az
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import display

from rheojax.core.jax_config import safe_import_jax, verify_float64
from rheojax.models.sgr import SGRConventional, SGRGeneric

jax, jnp = safe_import_jax()
verify_float64()

warnings.filterwarnings("ignore", category=FutureWarning)
print(f"JAX version: {jax.__version__}")
print(f"Devices: {jax.devices()}")

## 2. Theory: SGR Trap Model

The Soft Glassy Rheology (SGR) model describes materials as collections of mesoscopic elements trapped in energy wells of depth $E$. A noise temperature $x$ provides thermal-like kicks that enable rearrangements.

### Key Equations

**Hopping rate** (Arrhenius-like):
$$
\Gamma(E) = \tau_0^{-1} \exp\left(-\frac{E}{x}\right)
$$

**Steady-shear viscosity** (power-law):
$$
\eta(\dot{\gamma}) = G_0 \tau_0 \, \mathcal{G}(x) \, (\dot{\gamma} \tau_0)^{x-2}
$$
where $\mathcal{G}(x) = \Gamma(2-x) / \Gamma(1-x)$ is the Gamma function ratio.

### Phase Regimes

| Range | Regime | Viscosity | Example |
|-------|--------|-----------|----------|
| $x < 1$ | **Glass** (yield stress) | $\eta \sim \dot{\gamma}^{x-2}$ (strong shear-thinning) | Dense emulsions, foams |
| $1 \leq x < 2$ | **Power-law fluid** | $\eta \sim \dot{\gamma}^{x-2}$ (moderate shear-thinning) | Pastes, colloidal suspensions |
| $x \geq 2$ | **Newtonian** | $\eta \approx$ constant | Low-concentration dispersions |

### Parameters

- $x$ : Noise temperature (0.5–3, dimensionless) — controls phase behavior
- $G_0$ : Elastic modulus (Pa) — sets material stiffness
- $\tau_0$ : Microscopic attempt time (s) — molecular/colloidal relaxation scale

## 3. Load Data

We use six emulsion datasets at different volume fractions ($\phi = 0.69, 0.70, 0.72, 0.74, 0.76, 0.80$). Each CSV contains (Shear Rate, Stress) columns.

**Important:** SGR predicts **viscosity** $\eta(\dot{\gamma})$, not stress. We must convert: $\eta = \sigma / \dot{\gamma}$.

In [ ]:
# Load all 6 emulsion datasets and convert stress to viscosity
phi_values = [0.69, 0.70, 0.72, 0.74, 0.76, 0.80]
datasets = {}

for phi in phi_values:
    data_path = os.path.join("..", "data", "flow", "emulsions", f"{phi:.2f}.csv")
    raw = np.loadtxt(data_path, delimiter=",", skiprows=1)
    gamma_dot = raw[:, 0]
    stress = raw[:, 1]
    eta = stress / gamma_dot  # SGR predicts viscosity, not stress
    datasets[phi] = {"gamma_dot": gamma_dot, "stress": stress, "eta": eta}
    print(f"φ={phi:.2f}: {len(gamma_dot)} points, γ̇=[{gamma_dot.min():.4f}, {gamma_dot.max():.1f}] 1/s")

In [ ]:
# Plot all 6 flow curves (stress and viscosity)
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))
colors = plt.cm.viridis(np.linspace(0.2, 0.9, len(phi_values)))

for i, phi in enumerate(phi_values):
    d = datasets[phi]
    ax1.loglog(d["gamma_dot"], d["stress"], "o", color=colors[i], markersize=5, label=f"φ={phi:.2f}")
    ax2.loglog(d["gamma_dot"], d["eta"], "o", color=colors[i], markersize=5, label=f"φ={phi:.2f}")

ax1.set_xlabel("Shear rate [1/s]")
ax1.set_ylabel("Stress [Pa]")
ax1.set_title("Raw Flow Curves (Stress)")
ax1.legend(fontsize=8)
ax1.grid(True, alpha=0.3, which="both")

ax2.set_xlabel("Shear rate [1/s]")
ax2.set_ylabel("Viscosity [Pa·s]")
ax2.set_title("Flow Curves (Viscosity)")
ax2.legend(fontsize=8)
ax2.grid(True, alpha=0.3, which="both")

plt.tight_layout()
display(fig)
plt.close(fig)

## 4. NLSQ Fitting

### 4.1 Single Sample (φ=0.80)

We start by fitting the most concentrated emulsion ($\phi=0.80$), which should exhibit glass-like behavior ($x < 1$).

In [ ]:
# Fit SGR to φ=0.80 sample
model = SGRConventional()

gamma_dot_80 = datasets[0.80]["gamma_dot"]
eta_80 = datasets[0.80]["eta"]

t0 = time.time()
model.fit(gamma_dot_80, eta_80, test_mode="steady_shear")
t_nlsq = time.time() - t0

print(f"NLSQ fit time: {t_nlsq:.2f} s")
print(f"R²: {model._fit_result.r_squared:.6f}")
print(f"RMSE: {model._fit_result.rmse:.4g} Pa·s")
print("\nFitted parameters:")
for name in ["x", "G0", "tau0"]:
    val = model.parameters.get_value(name)
    print(f"  {name:5s} = {val:.4g}")

regime = model.get_phase_regime()
print(f"\nPhase regime: {regime}")

In [ ]:
# Plot fit with data
gamma_dot_fine = np.logspace(
    np.log10(gamma_dot_80.min()) - 0.5,
    np.log10(gamma_dot_80.max()) + 0.2,
    200,
)
eta_pred = model.predict(gamma_dot_fine)

fig, ax = plt.subplots(figsize=(8, 5))
ax.loglog(gamma_dot_80, eta_80, "ko", markersize=6, label="Data φ=0.80")
ax.loglog(gamma_dot_fine, eta_pred, "-", lw=2, color="C0", label="SGR fit")
ax.set_xlabel("Shear rate [1/s]")
ax.set_ylabel("Viscosity [Pa·s]")
ax.set_title(f"SGR Fit — x={model.parameters.get_value('x'):.3f} ({regime})")
ax.legend()
ax.grid(True, alpha=0.3, which="both")
plt.tight_layout()
display(fig)
plt.close(fig)

### 4.2 Volume Fraction Sweep

Now we fit all six volume fractions to observe how the noise temperature $x$ evolves as the emulsion approaches the glass transition.

In [ ]:
# Fit all 6 samples and track x(φ)
fit_results = {}

for phi in phi_values:
    m = SGRConventional()
    d = datasets[phi]
    m.fit(d["gamma_dot"], d["eta"], test_mode="steady_shear")
    fit_results[phi] = {
        "x": float(m.parameters.get_value("x")),
        "G0": float(m.parameters.get_value("G0")),
        "tau0": float(m.parameters.get_value("tau0")),
        "R2": float(m._fit_result.r_squared),
        "regime": m.get_phase_regime(),
    }

print(f"{'φ':>6s}  {'x':>6s}  {'G0':>10s}  {'τ₀':>10s}  {'R²':>8s}  {'Regime'}")
print("-" * 60)
for phi in phi_values:
    r = fit_results[phi]
    print(f"{phi:6.2f}  {r['x']:6.3f}  {r['G0']:10.2f}  {r['tau0']:10.2e}  {r['R2']:8.5f}  {r['regime']}")

In [ ]:
# Plot x(φ) trend and fit quality
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

phis = list(fit_results.keys())
x_vals = [fit_results[p]["x"] for p in phis]

ax1.plot(phis, x_vals, "o-", markersize=8, lw=2)
ax1.axhline(1.0, color="red", linestyle="--", alpha=0.5, label="Glass transition (x=1)")
ax1.axhline(2.0, color="blue", linestyle="--", alpha=0.5, label="Newtonian (x=2)")
ax1.set_xlabel("Volume fraction φ")
ax1.set_ylabel("Noise temperature x")
ax1.set_title("x(φ): Approach to Glass Transition")
ax1.legend()
ax1.grid(True, alpha=0.3)

# Phase regime color bar
regime_colors = {"glass": "red", "power-law": "orange", "newtonian": "blue"}
for i, phi in enumerate(phis):
    color = regime_colors[fit_results[phi]["regime"]]
    ax2.bar(phi, fit_results[phi]["R2"], width=0.015, color=color, alpha=0.7)
ax2.set_xlabel("Volume fraction φ")
ax2.set_ylabel("R²")
ax2.set_title("Fit Quality by Volume Fraction")
ax2.set_ylim(0.95, 1.005)
ax2.grid(True, alpha=0.3)

# Legend for regimes
from matplotlib.patches import Patch
legend_elements = [Patch(facecolor=c, alpha=0.7, label=r) for r, c in regime_colors.items()]
ax2.legend(handles=legend_elements, title="Phase regime")

plt.tight_layout()
display(fig)
plt.close(fig)

### 4.3 Shear Banding Detection

For concentrated emulsions with $x < 1$, the SGR model can predict **shear banding** — a mechanical instability where the material splits into coexisting bands of different shear rates.

In [ ]:
# Detect shear banding for φ=0.80
model_80 = SGRConventional()
model_80.fit(datasets[0.80]["gamma_dot"], datasets[0.80]["eta"], test_mode="steady_shear")

is_banding, banding_info = model_80.detect_shear_banding()
print(f"Shear banding detected: {is_banding}")
if banding_info:
    print(f"Banding info: {banding_info}")
else:
    x_val = model_80.parameters.get_value("x")
    print(f"No banding (x={x_val:.3f} — banding requires x < 1 typically)")

### 4.4 SGRGeneric Comparison

The **SGRGeneric** model implements the GENERIC (General Equation for the Non-Equilibrium Reversible-Irreversible Coupling) framework, ensuring thermodynamic consistency.

In [ ]:
# Compare SGRConventional vs SGRGeneric on φ=0.80
model_gen = SGRGeneric()
model_gen.fit(gamma_dot_80, eta_80, test_mode="steady_shear")

print("SGRConventional vs SGRGeneric:")
print(f"  Conventional: x={model.parameters.get_value('x'):.4f}, R²={model._fit_result.r_squared:.6f}")
print(f"  Generic:      x={model_gen.parameters.get_value('x'):.4f}, R²={model_gen._fit_result.r_squared:.6f}")

# Thermodynamic consistency check
state = np.array([100.0, 0.5])
consistency = model_gen.verify_thermodynamic_consistency(state)
print(f"\nThermodynamic consistency: {consistency.get('thermodynamically_consistent', 'N/A')}")

# Plot comparison
eta_conv_pred = model.predict(gamma_dot_fine)
eta_gen_pred = model_gen.predict(gamma_dot_fine)

fig, ax = plt.subplots(figsize=(8, 5))
ax.loglog(gamma_dot_80, eta_80, "ko", markersize=6, label="Data")
ax.loglog(gamma_dot_fine, eta_conv_pred, "-", lw=2, label="Conventional")
ax.loglog(gamma_dot_fine, eta_gen_pred, "--", lw=2, label="Generic (GENERIC)")
ax.set_xlabel("Shear rate [1/s]")
ax.set_ylabel("Viscosity [Pa·s]")
ax.set_title("SGR Model Comparison (φ=0.80)")
ax.legend()
ax.grid(True, alpha=0.3, which="both")
plt.tight_layout()
display(fig)
plt.close(fig)

## 5. Bayesian Inference with NUTS

### 5.1 Run NUTS

We use the NLSQ point estimates as warm-start values for efficient MCMC sampling with the No-U-Turn Sampler (NUTS).

In [ ]:
# Bayesian inference with NLSQ warm-start
initial_values = {
    name: model.parameters.get_value(name)
    for name in ["x", "G0", "tau0"]
}
print("Warm-start values:", initial_values)

# Fast demo config
NUM_WARMUP = 200
NUM_SAMPLES = 500
NUM_CHAINS = 1
# NUM_WARMUP = 1000; NUM_SAMPLES = 2000; NUM_CHAINS = 4  # production

t0 = time.time()
result = model.fit_bayesian(
    gamma_dot_80,
    eta_80,
    test_mode="steady_shear",
    num_warmup=NUM_WARMUP,
    num_samples=NUM_SAMPLES,
    num_chains=NUM_CHAINS,
    initial_values=initial_values,
    seed=42,
)
t_bayes = time.time() - t0
print(f"\nBayesian inference time: {t_bayes:.1f} s")

### 5.2 Convergence Diagnostics

Check R-hat (target < 1.01) and effective sample size (ESS, target > 400 per chain).

In [ ]:
# Convergence diagnostics table
diag = result.diagnostics
param_names = ["x", "G0", "tau0"]

print("Convergence Diagnostics")
print("=" * 50)
print(f"{'Parameter':>10s}  {'R-hat':>8s}  {'ESS':>8s}")
print("-" * 50)
for p in param_names:
    r_hat = diag.get("r_hat", {}).get(p, float("nan"))
    ess = diag.get("ess", {}).get(p, float("nan"))
    print(f"{p:>10s}  {r_hat:8.4f}  {ess:8.0f}")

n_div = diag.get("divergences", diag.get("num_divergences", 0))
print(f"\nDivergences: {n_div}")

### 5.3 ArviZ Diagnostic Plots

In [ ]:
# Trace plots
idata = result.to_inference_data()

axes = az.plot_trace(idata, var_names=param_names, figsize=(12, 6))
fig = axes.ravel()[0].figure
fig.suptitle("Trace Plots", fontsize=14, y=1.02)
plt.tight_layout()
display(fig)
plt.close(fig)

In [ ]:
# Pair plot with divergences
axes = az.plot_pair(
    idata,
    var_names=param_names,
    kind="scatter",
    divergences=True,
    figsize=(9, 9),
)
fig = axes.ravel()[0].figure
fig.suptitle("Parameter Correlations", fontsize=14, y=1.02)
plt.tight_layout()
display(fig)
plt.close(fig)

In [ ]:
# Forest plot (credible intervals)
axes = az.plot_forest(
    idata,
    var_names=param_names,
    combined=True,
    hdi_prob=0.95,
    figsize=(10, 4),
)
fig = axes.ravel()[0].figure
plt.tight_layout()
display(fig)
plt.close(fig)

### 5.4 Posterior Predictive Check

Validate the model by comparing data against predictions from posterior samples.

In [ ]:
# Posterior predictive with 95% credible intervals
posterior = result.posterior_samples
n_draws = min(200, len(list(posterior.values())[0]))
gamma_dot_pred = np.logspace(
    np.log10(gamma_dot_80.min()) - 0.3,
    np.log10(gamma_dot_80.max()) + 0.2,
    100,
)

pred_samples = []
for i in range(n_draws):
    params_i = jnp.array([posterior["x"][i], posterior["G0"][i], posterior["tau0"][i]])
    pred_i = model.model_function(jnp.array(gamma_dot_pred), params_i, test_mode="steady_shear")
    pred_samples.append(np.array(pred_i))

pred_samples = np.array(pred_samples)
pred_median = np.median(pred_samples, axis=0)
pred_lo = np.percentile(pred_samples, 2.5, axis=0)
pred_hi = np.percentile(pred_samples, 97.5, axis=0)

fig, ax = plt.subplots(figsize=(9, 6))
ax.fill_between(gamma_dot_pred, pred_lo, pred_hi, alpha=0.3, color="C0", label="95% CI")
ax.loglog(gamma_dot_pred, pred_median, "-", lw=2, color="C0", label="Posterior median")
ax.loglog(gamma_dot_80, eta_80, "ko", markersize=6, label="Data")
ax.set_xlabel("Shear rate [1/s]")
ax.set_ylabel("Viscosity [Pa·s]")
ax.set_title("Posterior Predictive Check")
ax.legend()
ax.grid(True, alpha=0.3, which="both")
plt.tight_layout()
display(fig)
plt.close(fig)

### 5.5 Parameter Summary

Compare NLSQ point estimates with Bayesian posterior medians and 95% credible intervals.

In [ ]:
# NLSQ vs Bayesian comparison table
print("Parameter Comparison: NLSQ vs Bayesian")
print("=" * 60)
print(f"{'Param':>8s}  {'NLSQ':>12s}  {'Bayes median':>14s}  {'95% CI':>20s}")
print("-" * 60)

for name in param_names:
    nlsq_val = model.parameters.get_value(name)
    bayes_samples = posterior[name]
    median = float(np.median(bayes_samples))
    lo = float(np.percentile(bayes_samples, 2.5))
    hi = float(np.percentile(bayes_samples, 97.5))
    print(f"{name:>8s}  {nlsq_val:12.4g}  {median:14.4g}  [{lo:.4g}, {hi:.4g}]")

## 6. Phase Regime Classification

The noise temperature $x$ determines the material's phase behavior:

| Phase Regime | $x$ Range | Viscosity Behavior | Material Examples |
|--------------|-----------|-------------------|-------------------|
| **Glass** | $x < 1$ | Yield stress + strong shear-thinning ($\eta \sim \dot{\gamma}^{x-2}$, $x-2 < -1$) | Dense emulsions ($\phi > 0.64$), concentrated foams, hard-sphere colloidal glasses |
| **Power-law fluid** | $1 \leq x < 2$ | Moderate shear-thinning ($\eta \sim \dot{\gamma}^{x-2}$, $-1 < x-2 < 0$) | Pastes, colloidal suspensions, polymer solutions |
| **Newtonian** | $x \geq 2$ | Constant viscosity ($\eta \approx$ const) | Dilute dispersions, simple fluids |

For the emulsion datasets:
- $\phi = 0.80$: $x \approx 0.7$ → **Glass** (jammed)
- $\phi = 0.69$: $x \approx 1.5$ → **Power-law fluid** (unjammed)

The transition occurs near random close packing ($\phi_c \approx 0.64$ for monodisperse spheres).

## 7. Save Results

In [ ]:
# Save results for downstream notebooks
output_dir = os.path.join("..", "outputs", "sgr", "flow_curve")
os.makedirs(output_dir, exist_ok=True)

# Save NLSQ point estimates (used by NB 05 and 06)
nlsq_params = {
    name: float(model.parameters.get_value(name))
    for name in ["x", "G0", "tau0"]
}
with open(os.path.join(output_dir, "nlsq_params.json"), "w") as f:
    json.dump(nlsq_params, f, indent=2)

# Save all volume fraction results
with open(os.path.join(output_dir, "phi_sweep_results.json"), "w") as f:
    json.dump({str(k): v for k, v in fit_results.items()}, f, indent=2)

# Save posterior samples
posterior_dict = {k: np.array(v).tolist() for k, v in posterior.items()}
with open(os.path.join(output_dir, "posterior_samples.json"), "w") as f:
    json.dump(posterior_dict, f)

print(f"Results saved to {output_dir}/")
print(f"  nlsq_params.json: {len(nlsq_params)} parameters (used by NB 05, 06)")
print(f"  phi_sweep_results.json: {len(fit_results)} volume fractions")
print(f"  posterior_samples.json: {n_draws} draws")

## Key Takeaways

1. **SGR predicts viscosity** $\eta(\dot{\gamma}) \sim (\dot{\gamma}\tau_0)^{x-2}$ — the shear-thinning exponent $x-2$ encodes the phase behavior.

2. **Increasing volume fraction** $\phi$ drives the noise temperature $x$ toward 1 (glass transition) as emulsions jam.

3. **Phase classification** via `get_phase_regime()`:
   - Glass ($x < 1$): Yield stress, strong shear-thinning
   - Power-law ($1 \leq x < 2$): Moderate shear-thinning
   - Newtonian ($x \geq 2$): Constant viscosity

4. **SGRGeneric** provides thermodynamically consistent predictions (GENERIC framework) with comparable fit quality to the conventional model.

5. **NLSQ warm-start is critical** for efficient Bayesian inference — enables rapid convergence with low R-hat and high ESS.

6. **Shear banding detection** identifies mechanical instabilities in glassy materials ($x < 1$).

### Next Steps

- **NB 02**: Stress relaxation (oscillatory cage dynamics)
- **NB 03**: SAOS moduli (frequency-dependent viscoelasticity)
- **NB 05**: Startup transients (stress overshoot from thixotropy)
- **NB 06**: LAOS harmonics (nonlinear response)